<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_8_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,13,up all night
1,16,couldn't taste anything
2,74,useless without
3,119,anxiety
4,154,urticaria


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-8.test.csv",header=None)
test.head()

,0,1
0,24,heart race 100mph
1,94,anxiety
2,154,#hivewatch
3,195,ballooning
4,195,bloated


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.205436,4.648635,0.231958,16:39


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.521110,4.354964,0.259103,16:43


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.238485,4.319652,0.262694,16:39


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.181355,4.293756,0.264946,16:44


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.136741,6.515196,0.097222,00:25


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.241637,5.532512,0.125000,00:30


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.723621,5.177565,0.118056,01:14


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.424408,4.832139,0.152778,01:19
1,5.232556,4.673085,0.187500,01:14


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.989051,4.510049,0.166667,01:12
1,4.795380,4.417491,0.194444,01:15


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.639442,4.297214,0.208333,01:15
1,4.493384,4.224164,0.208333,01:11


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.287727,4.061099,0.256944,01:17
1,4.156240,4.015263,0.291667,01:12


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.003902,4.002968,0.298611,01:21
1,3.854291,3.961858,0.291667,01:16


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.678904,3.882592,0.305556,01:18
1,3.635816,3.912874,0.291667,01:12
2,3.456796,3.902920,0.305556,01:15
3,3.310019,3.913775,0.319444,01:14


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.161218,3.840702,0.319444,01:20
1,3.134001,3.872542,0.326389,01:19
2,2.981421,3.863353,0.340278,01:20
3,2.839939,3.859670,0.333333,01:13


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.717860,3.885191,0.326389,01:18
1,2.696734,3.895324,0.333333,01:13
2,2.584040,3.967272,0.340278,01:14
3,2.420023,3.947171,0.340278,01:17


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.325504,3.978751,0.340278,01:18
1,2.297086,3.963191,0.347222,01:20
2,2.226658,4.111809,0.319444,01:13
3,2.115311,4.071815,0.333333,01:13


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.966659,4.063894,0.368056,01:17
1,2.017012,4.152583,0.340278,01:16
2,1.927825,4.118597,0.340278,01:17
3,1.857329,4.185721,0.326389,01:13


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.752220,4.229290,0.347222,01:14
1,1.772300,4.239222,0.333333,01:17
2,1.743869,4.343495,0.319444,01:15
3,1.635400,4.307464,0.312500,01:18


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.517801,4.294901,0.333333,01:17
1,1.628026,4.537681,0.333333,01:17
2,1.599334,4.485208,0.326389,01:13
3,1.540612,4.453173,0.326389,01:12


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[1230, 978, 1265, 1610, 1884, 224, 1936, 2147, 2147, 249, 1327, 1787, 1787, 1641, 1973, 1610, 1833, 1679, 852, 1227, 1610, 1610, 706, 1676, 744, 744, 1942, 1265, 1610, 749, 749, 195, 792, 792, 792, 2032, 233, 824, 1884, 1884, 265, 2010, 1295, 974, 978, 231, 749, 231, 187, 418, 393, 1099, 1610, 1099, 1610, 1136, 1610, 2023, 1610, 750, 1227, 1610, 1828, 1295, 1295, 1308, 1308, 1308, 1308, 1727, 1610, 1340, 1610, 1340, 1973, 1432, 1711, 1711, 1432, 1508, 852, 520, 2194, 1701, 1529, 1062, 561, 1529, 1973, 1573, 1597, 452, 60, 1597, 1597, 1597, 1597, 1531, 1196, 1610, 1597, 2191, 1767, 1073, 1798, 1767, 1718, 1718, 393, 2032, 393, 301, 1787, 393, 2032, 1815, 1610, 2149, 1936, 1546, 1940, 966, 1610, 1942, 1578, 2032, 1787, 1787, 1787, 189, 1295, 1769, 2103, 2149, 231, 1884, 2147, 1610, 1048, 1948, 1048, 1787]
[24, 94, 154, 195, 195, 224, 233, 233, 233, 249, 371, 393, 393, 402, 418, 426, 426, 441, 445, 561, 582, 645, 706, 720, 744, 744, 749, 749, 749, 749, 749, 767, 792, 792, 792, 792

In [44]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

44
0.30985915492957744
